In [7]:

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15150308560345827756
]


In [8]:
import tensorflow as tf
print(tf.__version__)

1.5.0


In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import time 
import scipy.io
import numpy as np
import tensorflow as tf
import h5py
import random
import matplotlib.pyplot as plt
from tensorflow.contrib import layers
from tensorflow.python.layers.core import Dense
import cv2
#print (cv2.__version__)

%matplotlib inline


In [8]:
def Architecture(Input, outputs, is_training=True):
    #Input,_,_=load_patch_data(Data_Train_Patch,heught=224,win_size=224,win_step=112,return_all=True)
    #Input.shape
    with tf.variable_scope("Network"):
        
        with tf.variable_scope("CNN"):
            #x_image = tf.reshape(image_batch, [-1, 64, 64, 3]) c moi besma, après on utilise x_image as son input
            conv1=tf.layers.conv2d(Input,64,3,activation=tf.nn.relu,name='conv1',padding='same')
            pool1=tf.layers.max_pooling2d(conv1,2,[2,2],name='pool1')
            #print('shape conv1',conv1.shape)
            #print('shape pool1 [2,2]',pool1.shape)
            #print('************************')
            
            conv2=tf.layers.conv2d(pool1,128,3,activation=tf.nn.relu,name='conv2',padding='same')
            pool2=tf.layers.max_pooling2d(conv2,2,[2,1],name='pool2')
           # print('shape conv2',conv2.shape)
           # print('shape pool2 [2,1]',pool1.shape)
           # print('************************')
            
            conv3=tf.layers.conv2d(pool2,256,3,activation=None,name='conv3',padding='same')
            b_norm3=tf.nn.relu(tf.layers.batch_normalization(conv3,training=is_training,name='batch-norm1'),name='b_norm3')
           # print('shape conv3',conv3.shape)
            #print('shape b_norm3',b_norm3.shape)
            #print('************************')
            
            conv4=tf.layers.conv2d(b_norm3,256,3,activation=tf.nn.relu,name='conv4',padding='same')
            pool4=tf.layers.max_pooling2d(conv4,2,[2,1],name='pool4',padding='same')
           # print('shape conv4',conv4.shape)
           # print('shape pool4 [2,1]',pool4.shape)
           # print('************************')
            
            conv5=tf.layers.conv2d(pool4,256,3,activation=tf.nn.relu,name='conv5',padding='same')
            pool5=tf.layers.max_pooling2d(conv5,2,[2,1],name='pool5',padding='same')
           # print('shape conv5',conv5.shape)
           # print('shape pool5 [2,1]',pool5.shape)
           # print('************************')
            
            conv6=tf.layers.conv2d(pool5,256,3,activation=None,name='conv6',padding='same')
            b_norm6=tf.nn.relu(tf.layers.batch_normalization(conv6,training=is_training,name='batch-norm2'),name='b_norm6')
           # print('shape conv6',conv6.shape)  
           # print('shape b_norm6',b_norm6.shape)
           # print('************************')
            
            conv7=tf.layers.conv2d(b_norm6,256,3,activation=tf.nn.relu,name='conv7',padding='same')
            pool7=tf.layers.max_pooling2d(conv7,2,[2,1],name='pool7',padding='same')
           # print('shape conv7',conv7.shape)
           # print('shape pool7[2,1]',pool7.shape)
           # print('************************')
            
            conv8=tf.layers.conv2d(pool7,256,2,activation=None,name='conv8')
            b_norm8=tf.nn.relu(tf.layers.batch_normalization(conv8,training=is_training,name='batch-norm3'),name='b_norm8')
           # print('shape conv8',conv8.shape)
           # print('shape b_norm8',b_norm8.shape)
           # print('************************')
            
            shape=b_norm8.get_shape().as_list()
            #print('shape of shape norm8',b_norm8.shape)#shape norm8 (32, 1, 30, 256)
            transposed=tf.transpose(b_norm8, perm=[0,2,1,3],name='transposed')
            conv_reshaped=tf.reshape(transposed, [shape[0], -1, shape[1]*shape[3]], name='reshaped')
            #print('shape conv_reshaped',conv_reshaped.shape)#shape conv_reshaped (32, 30, 256)
            
            
            
        num_units=256
        num_layers=3
        

       
        cell_stack_encoder=[]
        cell_stack_decoder=[]    
        
        with tf.variable_scope("Encoder"):
            for i in range(num_layers):
                cell_stack_encoder.append(tf.contrib.rnn.LSTMCell(num_units))
            encoder_cell=tf.contrib.rnn.MultiRNNCell(cell_stack_encoder, state_is_tuple=True)
            _, encoder_state=tf.nn.dynamic_rnn(encoder_cell, conv_reshaped, dtype=tf.float32) 
            print('encoder_state_shape',len(encoder_state))
              
        with tf.variable_scope("Decoder"):
            for i in range(num_layers):
                cell_stack_decoder.append(tf.contrib.rnn.LSTMCell(num_units))
            decoder_cell=tf.contrib.rnn.MultiRNNCell(cell_stack_decoder, state_is_tuple=True)
            dec_outputs, _=tf.nn.dynamic_rnn(decoder_cell, outputs, initial_state= encoder_state, dtype=tf.float32)
            print('dec_outputs', dec_outputs.get_shape())
            print('outputs=dec_inputs', outputs.get_shape())
            
            #logits=tf.layers.dense(dec_outputs,2, name='logits')
            logits=tf.layers.dense(dec_outputs,1, name='logits')
            
            return logits
            

In [10]:
def Architecture(Input, outputs, is_training=True):
    #Input,_,_=load_patch_data(Data_Train_Patch,heught=224,win_size=224,win_step=112,return_all=True)
    #Input.shape
    with tf.variable_scope("Network"):
        
        with tf.variable_scope("CNN"):
            #x_image = tf.reshape(image_batch, [-1, 64, 64, 3]) c moi besma, après on utilise x_image as son input
            conv1=tf.layers.conv2d(Input,64,3,activation=tf.nn.relu,name='conv1',padding='same')
            pool1=tf.layers.max_pooling2d(conv1,2,[2,2],name='pool1')
            #print('shape conv1',conv1.shape)
            #print('shape pool1 [2,2]',pool1.shape)
            #print('************************')
            
            conv2=tf.layers.conv2d(pool1,128,3,activation=tf.nn.relu,name='conv2',padding='same')
            pool2=tf.layers.max_pooling2d(conv2,2,[2,1],name='pool2')
           # print('shape conv2',conv2.shape)
           # print('shape pool2 [2,1]',pool1.shape)
           # print('************************')
            
            conv3=tf.layers.conv2d(pool2,256,3,activation=None,name='conv3',padding='same')
            b_norm3=tf.nn.relu(tf.layers.batch_normalization(conv3,training=is_training,name='batch-norm1'),name='b_norm3')
           # print('shape conv3',conv3.shape)
            #print('shape b_norm3',b_norm3.shape)
            #print('************************')
            
            conv4=tf.layers.conv2d(b_norm3,256,3,activation=tf.nn.relu,name='conv4',padding='same')
            pool4=tf.layers.max_pooling2d(conv4,2,[2,1],name='pool4',padding='same')
           # print('shape conv4',conv4.shape)
           # print('shape pool4 [2,1]',pool4.shape)
           # print('************************')
            
            conv5=tf.layers.conv2d(pool4,256,3,activation=tf.nn.relu,name='conv5',padding='same')
            pool5=tf.layers.max_pooling2d(conv5,2,[2,1],name='pool5',padding='same')
           # print('shape conv5',conv5.shape)
           # print('shape pool5 [2,1]',pool5.shape)
           # print('************************')
            
            conv6=tf.layers.conv2d(pool5,256,3,activation=None,name='conv6',padding='same')
            b_norm6=tf.nn.relu(tf.layers.batch_normalization(conv6,training=is_training,name='batch-norm2'),name='b_norm6')
           # print('shape conv6',conv6.shape)  
           # print('shape b_norm6',b_norm6.shape)
           # print('************************')
            
            conv7=tf.layers.conv2d(b_norm6,256,3,activation=tf.nn.relu,name='conv7',padding='same')
            pool7=tf.layers.max_pooling2d(conv7,2,[2,1],name='pool7',padding='same')
           # print('shape conv7',conv7.shape)
           # print('shape pool7[2,1]',pool7.shape)
           # print('************************')
            
            conv8=tf.layers.conv2d(pool7,256,2,activation=None,name='conv8')
            b_norm8=tf.nn.relu(tf.layers.batch_normalization(conv8,training=is_training,name='batch-norm3'),name='b_norm8')
           # print('shape conv8',conv8.shape)
           # print('shape b_norm8',b_norm8.shape)
           # print('************************')
            
            shape=b_norm8.get_shape().as_list()
            #print('shape of shape norm8',b_norm8.shape)#shape norm8 (32, 1, 30, 256)
            transposed=tf.transpose(b_norm8, perm=[0,2,1,3],name='transposed')
            conv_reshaped=tf.reshape(transposed, [shape[0], -1, shape[1]*shape[3]], name='reshaped')
            #print('shape conv_reshaped',conv_reshaped.shape)#shape conv_reshaped (32, 30, 256)
            
            
            
        num_units=256
        num_layers=3
        

       
        cell_stack_encoder=[]
        cell_stack_decoder=[]    
        
        with tf.variable_scope("Encoder"):
            for i in range(num_layers):
                cell_stack_encoder.append(tf.contrib.rnn.GRUCell(num_units))
           # encoder_cell=tf.contrib.rnn.MultiRNNCell(cell_stack_encoder, state_is_tuple=True)
            encoder_cell=tf.contrib.rnn.MultiRNNCell(cell_stack_encoder)
               # cell_stack_encoder.append(tf.contrib.rnn.LSTMCell(num_units))
           # encoder_cell=tf.contrib.rnn.MultiRNNCell(cell_stack_encoder, state_is_tuple=True)
            _, encoder_state=tf.nn.dynamic_rnn(encoder_cell, conv_reshaped, dtype=tf.float32) 
           
            #encoder_state = tuple(tf.concat([state_fw, state_bw], -1) for state_fw, state_bw in zip(encoder_state[0], encoder_state[1]))
           # print('encoder_state_shape',len(encoder_state))
             
            print('encoder_state', _.get_shape())
              
        with tf.variable_scope("Decoder"):
            for i in range(num_layers):
                cell_stack_decoder.append(tf.contrib.rnn.GRUCell(num_units))
            decoder_cell=tf.contrib.rnn.MultiRNNCell(cell_stack_decoder)
            
            dec_outputs,_=tf.nn.dynamic_rnn(decoder_cell,outputs, initial_state=encoder_state, dtype=tf.float32)
            
            #logits=tf.layers.dense(dec_outputs,2, name='logits')
            logits=tf.layers.dense(dec_outputs,1, name='logits')
            
            return logits
            

In [12]:
def load_Images(Data_Train_Path):
    #main_path=[f for f in np.sort(os.listdir(Data_Train_Path)) if f.endswith('redraw')]
    main_path=[f for f in np.sort(os.listdir(Data_Train_Path)) if f.endswith('redraw')]
    height =28
    imgs=[]
    for s in main_path:
        sub=Data_Train_Path+s
        sub_path= [f for f in np.sort(os.listdir(sub))if f.endswith('.tif')]
        for t in sub_path:
            im_path=sub + '/' +t
            img=cv2.imread(im_path,cv2.IMREAD_UNCHANGED)
            imgs.append(img)
            #plt.imshow(img)
            #plt.show()
           # print('name image',t)
    #return np.expand_dims(imgs, -1,f)
    return (np.expand_dims(imgs, -1),t)


##*****************************************************************************************************
#saver.restore(sess, tf.train.latest_checkpoint('./') )  
#Save_folder = '/home/razen/thèse_2018/second_contrib/code/Besma_code_Recovery/recover_IRONOFF_caracters/systeme_2018/data_train_test/New_test_40_echantillons/recovered_2018/aa/'
Save_folder ='/home/razen/thèse_2018/second_contrib/code/Besma_code_Recovery/recover_IRONOFF_Words/ironoffwords/test_word/recov/'
#Save_folder ='/home/razen/thèse_2018/second_contrib/code/Besma_code_Recovery/recover_IRONOFF_caracters/digit_2018/data_test/00/'
#model_train_path = "/home/razen/tf-notebooks/logs_L3_28_digits/logs_L3_28_digits.ckpt-1"
model_train_path = "/home/razen/tf-notebooks/logspenup_GRU_words3/logspenup_GRU_words3.ckpt-1"

#model_train_path = "/home/razen/tf-notebooks/logspenup_words/logspenup_words.ckpt-1"
#model_train_path = "/home/razen/tf-notebooks/model_logs_4_alv/logs_L4model_logs_L4_alv.ckpt-1"
number = 0
from tensorflow.python.framework import ops
ops.reset_default_graph()
tf.reset_default_graph()
global sess

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session (config=config)
graph = tf.get_default_graph()
import os
#my_DataTest = load_Images('/home/razen/tf-notebooks/New_ironoff_test_10_echantillons/')
my_DataTest,f = load_Images('/home/razen/thèse_2018/second_contrib/code/Besma_code_Recovery/recover_IRONOFF_Words/ironoffwords/test_word/')
#my_DataTest,f = load_Images('/home/razen/thèse_2018/second_contrib/code/Besma_code_Recovery/recover_IRONOFF_caracters/systeme_2018/data_train_test/New_test_40_echantillons/redraw/')
#_,f=load_Images('/home/razen/tf-notebooks/IRONOFF_Characters_test_results_40/test_40_char_ironoff/usr_0_redraw_classed/redraw/')
print('my_DataTest ',my_DataTest,f)


#my_LabelTest_Targets, my_LabelTest_Outputs = load_Coordinates ('/home/razen/tf-notebooks/IRONOFF_Characters_test_results/test_folder/usr_0_online_classed/')
[Tot_Dat, row, col, depth] = np.shape(my_DataTest)
#rand_idx = np.random.choice(Tot_Dat, 260, replace=False)
#rand_idx = np.random.choice(Tot_Dat, 40, replace=False)
#my_Data_Test = my_DataTest[rand_idx,:,:,:]
#for i_im in range(40):
#    my_Data_Test = my_DataTest[i_im,:,:,:]
#    print('my_Data_Test index',my_Data_Test.index)

#my_LabelTest_Outputs = my_LabelTest_Outputs[rand_idx,:,:]
#my_LabelTest_Targets = my_LabelTest_Targets[rand_idx,:,:]


BatchSize_Test = 2

Inputs = tf.placeholder(tf.float32, shape = [BatchSize_Test, 50, 150, 1],name = 'Inputs')
outputs = tf.placeholder(tf.float32, shape = [BatchSize_Test, None, 1],name = 'outputs')
#targets = tf.placeholder(tf.float32, shape = [BatchSize_Test, 50, 2],name = 'targets')

    
logits = Architecture(Inputs, outputs)
#loss = Loss_function(logits, targets)

#init = tf.global_variables_initializer()
#sess.run(init)
#saver.restore(sess, 'logs_L2model_logs_L2' )  
#############################################train accuracy#####################################################
#correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(targets, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#accuracy = np.mean(logits.argmax(axis=-1) == targets[:,1:])
################################################################################################################
#tf.reset_default_graph()
saver = tf.train.Saver()
with tf.Session() as sess:
    #meta = tf.train.import_meta_graph('logs_L2model_logs_L2.ckpt-1.meta')
    saver.restore(sess, model_train_path)
    #meta.restore(sess, tf.train.latest_checkpoint('./'))
    #saver.restore(sess, model_path)
    #print("Model restored from file: %s" % save_path)
    
    #saver.restore(sess, tf.train.latest_checkpoint('./') )  
    #meta.restore(sess, tf.train.latest_checkpoint('./')
                 
    
     #print(sess.run(Y, feed_dict={x_placeholder: X))


#for k in range(3):
    start = time.time()
    Test_loss = []

    
    for b in range(Tot_Dat//BatchSize_Test):
        batch_Test_input = my_DataTest[b*BatchSize_Test:(b + 1)*BatchSize_Test,:,:,:]
        print('b',b)
        print('debut b*BatchSize_Test',b*BatchSize_Test)
        print('fin batch_Test_input index',(b + 1)*BatchSize_Test)
        
        #batch_Test_targets = my_LabelTest_Targets[b*BatchSize_Test:(b + 1)*BatchSize_Test,:,:]
        batch_Test_outputs = np.zeros(shape = [BatchSize_Test,1,1])
                #batch_Test_input = Data_Test[6,:,:,:]
                #batch_Test_targets = LabelTest_Targets[6,:,:]
                #batch_Test_outputs = np.zeros(shape = [BatchSize_Test,1,2])
                    
        for ij in range(100):
                #test_feed_dict = {batch_Test_input, batch_Test_outputs}
            test_feed_dict = {Inputs:batch_Test_input, outputs:batch_Test_outputs}
            logits_Out = sess.run([logits], feed_dict = test_feed_dict)
            
            logits_Out = logits_Out[0]
                
            prediction = np.expand_dims(logits_Out[:,-1,:],axis = 1)
            batch_Test_outputs = np.hstack([batch_Test_outputs, prediction])
            #acc= sess.run([accuracy], feed_dict={logits: logits_Out,targets:batch_Test_targets})            
                #np.savetxt('logits_Out[0]', logits_Out[0])
                #print('logits_Out:',np.array(logits_Out))
                #print('logits_Out[0]:',np.array(logits_Out[0]))
                #print('logits_Out[1]:',np.array(logits_Out[1]))
                #print('logits_Out_shape:',np.array(logits_Out.shape))
        #loss_ = Loss_function_Test(logits_Out, batch_Test_targets)
        #loss_ = tf.losses.mean_squared_error(logits_Out, batch_Test_targets)
        #loss_=tf.metrics.root_mean_squared_error(logits_Out, batch_Test_targets)
         #loss_=tf.sqrt(tf.losses.mean_squared_error(logits_Out, batch_Test_targets))
        #file = open("./output/" + "logits_pred33.txt","a") 
        
        #file.write(str(logits_Out[0]) + "\t" +str(batch_Test_targets)+"\t" "\n")

       # file.close()
        #Test_loss.append(loss_)
        

        ############################################
        for num in range(2):
            #directory = Save_folder + str(number) + '/'
            directory = Save_folder 
            try:
                os.stat(directory)
            except:
                os.mkdir(directory) 
           # img = 255 - batch_Test_input[num, :,:,0]
            #imgpath = directory + str(number) + '.jpg'
            #cv2.imwrite(imgpath, img)
            #np.savetxt(  directory + 'Grountruth_' + str(num)+str(number) + '.txt', batch_Test_targets[num,:,:])
            logout_float=logits_Out[num,:,:]
            logout_int=logout_float.astype(int)
            
            np.savetxt(  directory + 'Predict_flaot' + str(num)+str(number) + '.txt', logout_float)
            np.savetxt(  directory + 'Predict_txt' + str(num)+str(number) + '.txt', logout_int)
            
            number = number + 1
        #if jk%3 == 0:
            #file.close()
            #reDraw(batch_Test_targets.astype(int), logits_Out.astype(int),2, ' Testing')
        ##########################################
    
    #print('### TESTING TESTING TESING ###')
    #print( 'Test Loss is: => ' + str(Test_loss) + '\n')
    #print('### TESTING TESTING TESING ###')

my_DataTest  [[[[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  ...

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]]


 [[[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  ...

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]]


 [[[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  ...

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]]] 61.tif
encoder_state (2, 73, 256)
INFO:tens

InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [64] rhs shape= [50]
	 [[Node: save/Assign_12 = Assign[T=DT_FLOAT, _class=["loc:@Network/CNN/conv1/bias"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Network/CNN/conv1/bias, save/RestoreV2_12)]]

Caused by op u'save/Assign_12', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/razen/.local/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/razen/.local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/razen/.local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/razen/.local/lib/python